In [1]:
import pandas as pd
import yaml
import logging

first we will read the file using the normal not dynamic method

In [2]:
data=pd.read_csv("C:/Users/Fawzi/Desktop/datasets.csv/games.csv")
data.head()
# our goal is to parameterize this reading method, this parameterization is doing through the yaml file. (we can use json as config file)

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.500000e+12,1.500000e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.500000e+12,1.500000e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.500000e+12,1.500000e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.500000e+12,1.500000e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.500000e+12,1.500000e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


##### our goal is to make this process of reading more dynamic by creating a config file. To do so we must clarify some utility functions that enable us to read and process our yaml file and making also some operations if needed.

In [3]:
# function to read our config file (yaml)
def read_config_file(filepath):
    with open (filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)
            


# let's define another function to validate the header of the file 

def col_header_val(df,table_config):
   
    
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed or we didn't call all of the features")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

#### Reading the file using our yaml configuration file:

In [4]:
# reading the yaml file using the function we defined above
config_data= read_config_file("C:/Users/Fawzi/Desktop/Data ingestion pipeline/file.yaml")
config_data
# we can see that our function allowed us to read the yaml file as dictionnary

{'file_type': 'csv',
 'dataset_name': 'chess',
 'file_name': 'games',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'columns': ['rated',
  'winner',
  'white_id',
  'black_id',
  'victory_status',
  'opening_name']}

In [5]:
file_type=config_data['file_type']
source_file= "C:/Users/Fawzi/Desktop/datasets.csv/" + config_data['file_name'] + f'.{file_type}'

df=pd.read_csv(source_file)
df[config_data['columns']]
# This cell is fixed, so if we want to read another dataset from our file and specific columns all we have to do is changing
# our yaml file and then run these two cells and we will get the new results 

,rated,winner,white_id,black_id,victory_status,opening_name
0,False,white,bourgris,a-00,outoftime,Slav Defense: Exchange Variation
1,True,black,a-00,skinnerua,resign,Nimzowitsch Defense: Kennedy Variation
2,True,white,ischia,a-00,mate,King's Pawn Game: Leonardis Variation
3,True,white,daniamurashov,adivanov2009,mate,Queen's Pawn Game: Zukertort Variation
4,True,white,nik221107,adivanov2009,mate,Philidor Defense
...,...,...,...,...,...,...
20053,True,white,belcolt,jamboger,resign,Dutch Defense
20054,True,black,jamboger,farrukhasomiddinov,mate,Queen's Pawn
20055,True,white,jamboger,schaaksmurf3,mate,Queen's Pawn Game: Mason Attack
20056,True,white,marcodisogno,jamboger,resign,Pirc Defense


#### finally we can simply change the inputs in the config file to read any dataset and columns we need from that folder without changing anything here --> hence, we turned our reading process to be worked dinamcally

In [6]:
col_header_val(df, config_data)

column name and column length validation failed or we didn't call all of the features
Following File columns are not in the YAML file ['black_rating', 'opening_ply', 'turns', 'opening_eco', 'last_move_at', 'increment_code', 'created_at', 'moves', 'id', 'white_rating']
Following YAML columns are not in the file uploaded []


0